In [15]:
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Dropout
from keras_tqdm import TQDMNotebookCallback
import keras.callbacks

from sqlalchemy import create_engine
from itertools import combinations
from operator import itemgetter
import json

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [33]:
version = 'v2'

race_type = 'R'
r = 2

bet_type = 'E'

In [7]:
engine = create_engine('sqlite:///../../data/race.db')
conn = engine.connect()
conn
dfoo = pd.read_sql_table('race', conn)
print('{} races loaded!'.format(len(dfoo)))

31867 races loaded!


In [8]:
dfo = dfoo.loc[dfoo['race_type'].isin([race_type])]
print('{} {} races!'.format(len(dfo), race_type))

10248 R races!


In [9]:
dfo['results'] = dfo['results_data'].map(json.loads)
dfo['runners'] = dfo['runners_data'].map(json.loads)
dfo.head(3)

/Users/jaco/.pyenv/versions/3.6.1/envs/tabby/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/jaco/.pyenv/versions/3.6.1/envs/tabby/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,id,meeting_name,location,venue_mnemonic,race_type,meeting_date,race_number,race_name,race_start_time,race_status,race_distance,results_data,num_runners,runners_data,quinella,exacta,trifecta,first_four,results,runners
0,1,BALLINA,NSW,C,R,2017-10-01,1,COATES HIRE MAIDEN PLATE,2017-10-01 01:40:00,Paying,1600,"[[4], [9], [7], [6]]",11.0,"[{""runnerName"": ""WHO AM I"", ""runnerNumber"": 12...",35.2,27.4,170.9,401.1,"[[4], [9], [7], [6]]","[{'runnerName': 'WHO AM I', 'runnerNumber': 12..."
1,3,BALLINA,NSW,C,R,2017-10-01,3,CNW ELECTRICAL WHOLESALE CG&E BM55,2017-10-01 02:55:00,Paying,1000,"[[3], [4], [5], [1]]",7.0,"[{""runnerName"": ""MCLENNAN"", ""runnerNumber"": 1,...",14.1,23.1,100.3,710.5,"[[3], [4], [5], [1]]","[{'runnerName': 'MCLENNAN', 'runnerNumber': 1,..."
2,4,BALLINA,NSW,C,R,2017-10-01,4,LEND LEASE MAIDEN HANDICAP,2017-10-01 03:35:00,Paying,1300,"[[14], [5], [1], [13]]",12.0,"[{""runnerName"": ""WOODDITTON"", ""runnerNumber"": ...",8.5,22.6,49.6,1158.9,"[[14], [5], [1], [13]]","[{'runnerName': 'WOODDITTON', 'runnerNumber': ..."


In [21]:
def build_combinations(runners, r, outcome=None, dividend=None):
    data = []
    # sort to combinations instead of permutations
    runners = sorted(runners, key=itemgetter('win_scaled'), reverse=True)
    # create combs for box of size 'r' runners
    combs = combinations(runners, r)
    # construct data object for every combination
    for comb in combs:
        item = {}
        # add every runner data in comb
        for i, runner in enumerate(comb):
            # only add combs with runners that have odds
            if not runner['has_odds']:
                break
            item.update({
                'run{}_num'.format(i + 1): runner['runnerNumber'],
                'run{}_win_perc'.format(i + 1): runner['win_perc'],
                'run{}_win_scaled'.format(i + 1): runner['win_scaled'],
                'run{}_win_rank'.format(i + 1): runner['win_rank'],
                'run{}_place_perc'.format(i + 1): runner['place_perc'],
                'run{}_place_scaled'.format(i + 1): runner['place_scaled'],
                'run{}_place_rank'.format(i + 1): runner['place_rank'],
            })
        # if broken then a runner does not have odds, and rest (incl add) should be in else
        else:
            # all runners data added, now add rest or meeting specific data
            item.update({
                'num_runners': 1 / runner['num_runners'],
                'race_type': race_type,
                'bet_type': bet_type,
            })
            if outcome and dividend:
                res1, res2, res3, res4 = outcome
                try:
                    res1, res2, res3, res4 = res1[0], res2[0], res3[0], res4[0]
                except IndexError as e:
                    continue
                item.update({
                    'success': 1 if item['run1_num'] == res1 and item['run2_num'] == res2 else 0,
                    'dividend': dividend,
                })
            data.append(item)
    return data

In [28]:
# extract runners from races
data_all = pd.DataFrame()
for ri, race in dfo.iterrows():
    combs = build_combinations(race['runners'], r, race['results'], race['exacta'])
#     print(combs)
    data_all = data_all.append(combs)
    break
data_all.tail()

,bet_type,dividend,num_runners,race_type,run1_num,run1_place_perc,run1_place_rank,run1_place_scaled,run1_win_perc,run1_win_rank,run1_win_scaled,run2_num,run2_place_perc,run2_place_rank,run2_place_scaled,run2_win_perc,run2_win_rank,run2_win_scaled,success
50,Q,27.4,11.0,R,11,0.100000,0.416667,0.027687,0.018215,0.416667,0.015117,3,0.038911,0.166667,0.010773,0.008554,0.250000,0.007099,0
51,Q,27.4,11.0,R,11,0.100000,0.416667,0.027687,0.018215,0.416667,0.015117,2,0.041322,0.250000,0.011441,0.008410,0.166667,0.006980,0
52,Q,27.4,11.0,R,5,0.087719,0.333333,0.024287,0.011099,0.333333,0.009211,3,0.038911,0.166667,0.010773,0.008554,0.250000,0.007099,0
53,Q,27.4,11.0,R,5,0.087719,0.333333,0.024287,0.011099,0.333333,0.009211,2,0.041322,0.250000,0.011441,0.008410,0.166667,0.006980,0
54,Q,27.4,11.0,R,3,0.038911,0.166667,0.010773,0.008554,0.250000,0.007099,2,0.041322,0.250000,0.011441,0.008410,0.166667,0.006980,0


In [30]:
# get label data
Y = data_all['success']

#print(data['finishingPosition'].head(10))
Y = Y.astype(int)
# Y.head(10)
Y.describe()

count    55.000000
mean      0.018182
std       0.134840
min       0.000000
25%       0.000000
50%       0.000000
75%       0.000000
max       1.000000
Name: success, dtype: float64

In [31]:
X = data_all[[
    'num_runners',
    'run1_win_perc', 'run1_win_scaled', 'run1_win_rank', 'run1_place_perc', 'run1_place_scaled', 'run1_place_rank',
    'run2_win_perc', 'run2_win_scaled', 'run2_win_rank', 'run2_place_perc', 'run2_place_scaled', 'run2_place_rank'
]]
X.tail()

,num_runners,run1_win_perc,run1_win_scaled,run1_win_rank,run1_place_perc,run1_place_scaled,run1_place_rank,run2_win_perc,run2_win_scaled,run2_win_rank,run2_place_perc,run2_place_scaled,run2_place_rank
50,11.0,0.018215,0.015117,0.416667,0.100000,0.027687,0.416667,0.008554,0.007099,0.250000,0.038911,0.010773,0.166667
51,11.0,0.018215,0.015117,0.416667,0.100000,0.027687,0.416667,0.008410,0.006980,0.166667,0.041322,0.011441,0.250000
52,11.0,0.011099,0.009211,0.333333,0.087719,0.024287,0.333333,0.008554,0.007099,0.250000,0.038911,0.010773,0.166667
53,11.0,0.011099,0.009211,0.333333,0.087719,0.024287,0.333333,0.008410,0.006980,0.166667,0.041322,0.011441,0.250000
54,11.0,0.008554,0.007099,0.250000,0.038911,0.010773,0.166667,0.008410,0.006980,0.166667,0.041322,0.011441,0.250000


In [34]:
# For a single-input model with 2 classes (binary classification):

n = len(X.columns)
print('input dimension = {}'.format(n))

epochs = 100
print('epochs = {}'.format(epochs))

layer_1 = 40
layer_2 = 40

tag = '{}{}x{}{}'.format(race_type, layer_1, layer_2, bet_type)
print('tag = {}'.format(tag))
file_name = '/Users/jaco/code/tabby/exotic/{}/models/{}.h5'.format(version, tag)

try:
    model = load_model(file_name)
    print('model loaded')
except OSError:
    model = Sequential()
    model.add(Dense(layer_1, activation='relu', input_dim=n))
    model.add(Dense(layer_2, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    print('model created')

input dimension = 13
epochs = 100
tag = R40x40E
model created


In [25]:
# Train the model, iterating on the data in batches of 32 samples
tqdm = TQDMNotebookCallback()

tbCallBack = keras.callbacks.TensorBoard(
    log_dir='/Users/jaco/code/tabby/exotic/{}/summary/{}'.format(version, tag), 
    histogram_freq=0,
    write_graph=True,
    write_images=True)

model.fit(
    X.as_matrix(), 
    Y.as_matrix(),
    validation_split=0.2,
    shuffle=True,
    epochs=epochs,
    batch_size=32,
    verbose=0,
    callbacks=[tqdm, tbCallBack])

# creates a HDF5 file
model.save(file_name)
print('model saved')


model saved
